In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from feature_selector import FeatureSelector

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
ind_categorical = ['DeviceType','DeviceInfo','id_13','id_14','id_17','id_18'
                  ,'id_19','id_20','id_30','id_31','id_32','id_33','id_38']
tr_categorical = ['ProductCD','card1','card2','card3','card4','card5','card6','addr1', 
                  'P_emaildomain','R_emaildomain','M3','M4','M5','M6']

In [4]:
coll = ['TransactionDT','TransactionAmt','ProductCD','card1','card2','card3','card4','card5','card6',
           'addr1','dist1','dist2','P_emaildomain','R_emaildomain','C1','C2','C4','C5','C6','C7','C8','C9',
           'C10','C11','C12','C13','C14','D1','D2','D4','D6','D8','D9','D10','D11','D12','D13','D15','M3','M4',
           'M5','M6','V3','V12','V13','V20','V34','V36','V38','V39','V44','V45','V52','V54','V55','V61','V62',
           'V66','V67','V76','V77','V78','V81','V82','V83','V85','V86','V87','V99','V102','V129','V130','V131',
           'V134','V136','V139','V149','V151','V154','V156','V161','V165','V187','V189','V200','V203','V207','V232',
           'V258','V261','V262','V265','V266','V272','V274','V281','V282','V283','V285','V294','V296','V301',
           'V308','V309','V310','V311','V312','V313','V314','V315','V316','V317','V320','id_01','id_02','id_03',
           'id_05','id_06','id_09','id_13','id_14','id_17','id_18','id_19','id_20','id_30','id_31','id_32',
           'id_33','id_38','DeviceType','DeviceInfo']

In [5]:
def nan(df):
    for column in df:
        if df[column].isnull().any():
               if df[column].dtypes == 'object' or column in ind_categorical:
                    values = df[column].value_counts().keys().tolist()
                    counts = df[column].value_counts().tolist()
                    if len(counts)!=0:
                        maxx = max(counts)
                        ind  = counts.index(maxx)
                        df[column] = df[column].fillna(values[ind])
               else:
                        df[column] = df[column].fillna(df[column].mean())
    return df
    del values, counts, maxx, ind, df


In [6]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
    

In [7]:
train_identity = pd.read_csv('./input/train_identity.csv')
test_identity = pd.read_csv('./input/test_identity.csv')
train_transaction = pd.read_csv('./input/train_transaction.csv')
test_transaction = pd.read_csv('./input/test_transaction.csv')
train_transaction = pd.merge(train_transaction, train_identity, on='TransactionID',how='left')
test_transaction = pd.merge(test_transaction, test_identity, on='TransactionID',how='left')

In [8]:
ind_train = train_transaction['TransactionID']
ind_test = test_transaction['TransactionID']
new_df = train_transaction.append(test_transaction).reset_index()

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
del train_identity, test_identity, train_transaction, test_transaction

In [10]:
def cols_to_drop(df,coll):
    cols_to_drop = [col for col in df.columns if col not in coll]
    cols_to_drop.remove('isFraud')
    cols_to_drop.remove('TransactionID')
    df = df.drop(cols_to_drop, axis=1)
    
    
    return df
    del cols_to_drop, coll, df

In [11]:
new_df = cols_to_drop(new_df, coll)

In [12]:
del coll

In [13]:
def feature_selector(df):
    fs_train = FeatureSelector(data = df, labels = df['isFraud'])
    fs_train.identify_missing(missing_threshold = 0.90)
    big_top_value_cols = [col for col in df.columns if df[col].value_counts(dropna=False, normalize=True).values[0] > 0.90]
    missing_features_train = fs_train.ops['missing']
    cols_drop = list(set(big_top_value_cols  +missing_features_train ))
    #cols_drop.remove('isFraud')
    df = df.drop(cols_drop, axis=1)
    return df
    del fs_train, big_top_value_cols, missing_features_train, cols_drop, df
    

In [14]:
new_df = feature_selector(new_df)

2 features with greater than 0.90 missing values.



In [15]:
new_df.shape

(1097231, 122)

In [16]:
train_transaction = new_df.loc[new_df['TransactionID'].isin(ind_train)]
test_transaction = new_df.loc[new_df['TransactionID'].isin(ind_test)]

In [17]:
del new_df

In [18]:
train_transaction = nan(train_transaction)
test_transaction = nan(test_transaction)

In [19]:
def features_engineering(df):
    print('Starting features engineering...')
    print('********************************')
    print('Number of features:',df.shape[1])
    print('********************************')
    print('Features D:')
    print('********************************')
    
    for col in df[['D1','D2','D4','D6','D8','D9','D10','D11','D12','D13']].columns:
        df[col+'_to_'+col+'mean_group_card1'] = df[col]/df.groupby(['card1'])[col].transform('mean')
        
    print('Number of features:',df.shape[1])
    print('Features D sample 2:')
    print('*********************************************************')
    
    for col in df[['D1','D2','D4','D8','D9','D10','D11']].columns:
            for col_card in df[['card2','card3','card4','card5','card6']].columns:
                df[col+'_to_'+col+'mean_group_'+col_card] = df[col]/df.groupby([col_card])[col].transform('mean')
                
    
    print('Number of features:',df.shape[1])
    print('Features TransactionDT:')
    print('********************************') 
    df['TransactionDT_minutes'] = df['TransactionDT'] / 60
    df['TransactionDT_hours'] = df['TransactionDT_minutes'] / 60
    df['TransactionDT_days'] = df['TransactionDT_hours'] / 24
    df['Transaction_day_of_week'] = round((df['TransactionDT'] / (3600 * 24) - 1) % 8)
    df['month'] = round(((df['TransactionDT_days']*365)/30)%12)
    
    print('Number of features:',df.shape[1])
    print('Features TransactionAmt:')
    print('********************************')
    
    for col in df[['ProductCD','month','Transaction_day_of_week','card1','card2','card3','card4','card5','card6']].columns:
        for transf in ['sum','mean','std']:
            if transf == 'sum':
                df['TransactionAmt_to_TransactionAmt'+transf.upper()+'_group_'+col] = df['TransactionAmt']/np.log(df.groupby([col])['TransactionAmt'].transform(transf))
            else:
                df['TransactionAmt_to_TransactionAmt'+transf.upper()+'_group_'+col] = df['TransactionAmt']/df.groupby([col])['TransactionAmt'].transform(transf)
        
    print('Number of features:',df.shape[1])
    print('Features TransactionDT:')
    print('********************************')
    
    for col in df[['card1','card2','card3','card4','card5','card6']].columns:
        for transf in ['sum','mean','std']:
            if transf == 'sum':
                df['TransactionDT_to_TransactionDT'+transf.upper()+'_group_'+col]   = np.log(df['TransactionDT'])/np.log(df.groupby([col])['TransactionDT'].transform(transf))
            else:    
                df['TransactionDT_to_TransactionDT'+transf.upper()+'_group_'+col]  = df['TransactionDT']/df.groupby([col])['TransactionDT'].transform(transf)
            
                                   
    print('Number of features:',df.shape[1]) 
    print('Features [DeviceInfo,id_30,id_31,id_33]:')
    print('********************************') 

    dict_split = {}
    dict_name = {}                              
    dict_split['DeviceInfo'] = '/'
    dict_name['DeviceInfo']  = ['NameDevice','VersionDevice']                            
    dict_split['id_30'] = ' '
    dict_name['id_30']  = ['OS','OS_Version']                          
    dict_split['id_31'] = ' '
    dict_name['id_31']  = ['NameBrowser','VerBrowser']                         
    dict_split['id_33'] = 'x'
    dict_name['id_33']  = ['width','height']                               
                    
    for col in df[['DeviceInfo','id_30','id_31','id_33']].columns:                     
            df[dict_name[col][0]]=df[col].str.split(dict_split[col], expand=True)[0]
            df[dict_name[col][1]]=df[col].str.split(dict_split[col], expand=True)[1]    
                                                              
                                   
    print('Number of features:',df.shape[1])
    print('Features C:')
    print('********************************')   
                                   
    for col in df[['C1','C2','C5','C6','C9','C11','C13','C14']].columns:
         for col_card in df[['card2','card3','card4','card5','card6']].columns:
            df[col+'_to_'+col+'Mean_group_'+col_card] = df[col]/df.groupby([col_card])[col].transform('mean')
            
    print('Number of features:',df.shape[1])
    print('Features P and R emaildomain:')
    print('********************************') 
                                   
    for col in df[['P_emaildomain','R_emaildomain']].columns:
        for transf in ['sum','mean','std']:
            if transf == 'sum':
                df['TransactionAmt_to_TransactionAmt'+transf.upper()+'_group_'+col] = np.log(df['TransactionAmt'])/np.log(df.groupby([col])['TransactionAmt'].transform(transf))
            else:    
                df['TransactionAmt_to_TransactionAmt'+transf.upper()+'_group_'+col] = df['TransactionAmt']/df.groupby([col])['TransactionAmt'].transform(transf)
                
            
    
    print('Number of features:',df.shape[1])
    print('VerBrowser and OS_Version:')
    print('********************************') 
    for col in df[['id_01','id_02']].columns:
        for col_2 in df[['VerBrowser','OS_Version']].columns:
            
            df[col+'_to_'+col+'Std_group_'+col_2] = df[col]/df.groupby([col_2])[col].transform('std')   
            df[col+'_to_'+col+'Mean_group_'+col_2] = df[col]/df.groupby([col_2])[col].transform('mean')
            
            if col_2=='OS_Version':
                df[col+'_to_'+col+'Mean_group_'+col_2] = np.log(abs(df[col]))/np.log(abs(df.groupby([col_2])[col].transform('sum')))
            else:    
                df[col+'_to_'+col+'Mean_group_'+col_2] = df[col]/df.groupby([col_2])[col].transform('sum') 
            
            
                                   
    print('Number of features:',df.shape[1])
    print('id_01 and id_02:')
    print('********************************')           
    for col in df[['id_13','id_14','id_17','id_19','id_20','id_38']].columns:
        for col_2 in df[['id_01','id_02']].columns:
            df[col_2+'_to_'+col_2+'Mean_group_'+col]= df[col_2]/df.groupby([col])[col_2].transform('mean') 
    print('Number of features:',df.shape[1])
    print('Stop features engineering...')
    return df

In [20]:
%%time
train_transaction = features_engineering(train_transaction)
test_transaction = features_engineering(test_transaction)

Starting features engineering...
********************************
Number of features: 122
********************************
Features D:
********************************
Number of features: 132
Features D sample 2:
*********************************************************
Number of features: 167
Features TransactionDT:
********************************
Number of features: 172
Features TransactionAmt:
********************************
Number of features: 199
Features TransactionDT:
********************************
Number of features: 217
Features [DeviceInfo,id_30,id_31,id_33]:
********************************
Number of features: 225
Features C:
********************************
Number of features: 265
Features P and R emaildomain:
********************************
Number of features: 271
VerBrowser and OS_Version:
********************************


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: RuntimeWarning: divide by zero encountered in log


Number of features: 279
id_01 and id_02:
********************************
Number of features: 291
Stop features engineering...
Starting features engineering...
********************************
Number of features: 122
********************************
Features D:
********************************
Number of features: 132
Features D sample 2:
*********************************************************
Number of features: 167
Features TransactionDT:
********************************
Number of features: 172
Features TransactionAmt:
********************************
Number of features: 199
Features TransactionDT:
********************************
Number of features: 217
Features [DeviceInfo,id_30,id_31,id_33]:
********************************
Number of features: 225
Features C:
********************************
Number of features: 265
Features P and R emaildomain:
********************************
Number of features: 271
VerBrowser and OS_Version:
********************************
Number of features: 

In [21]:
train_transaction = nan(train_transaction)
test_transaction = nan(test_transaction)

In [22]:
def quartile(df):
    # Calculate first and third quartile
    first_quartile = df['TransactionAmt'].describe()['25%']
    third_quartile = df['TransactionAmt'].describe()['75%']
    
    # Interquartile range
    iqr = third_quartile - first_quartile
    
    # Remove outliers
    df = df[(df['TransactionAmt'] > (first_quartile - 3 * iqr)) &
                (df['TransactionAmt'] < (third_quartile + 3 * iqr))]
    return df


In [23]:
train_transaction = quartile(train_transaction)

In [24]:
del ind_test, ind_train

In [25]:
test_transaction = test_transaction.drop(['isFraud'], axis=1)

In [26]:
train_transaction.shape, test_transaction.shape

((554118, 291), (506691, 290))

In [27]:
for col in tr_categorical:
    ind_categorical.append(col)

In [28]:
cat = []
for col in train_transaction.columns:
    if train_transaction[col].dtypes == 'object' and col not in ind_categorical:
        ind_categorical.append(col)  
ind_categorical

['DeviceType',
 'DeviceInfo',
 'id_13',
 'id_14',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_38',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'P_emaildomain',
 'R_emaildomain',
 'M3',
 'M4',
 'M5',
 'M6',
 'NameDevice',
 'VersionDevice',
 'OS',
 'OS_Version',
 'NameBrowser',
 'VerBrowser',
 'width',
 'height']

In [29]:
from sklearn import preprocessing
for col in test_transaction.columns:
    if  test_transaction[col].dtypes == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(test_transaction[col].astype(str))
        test_transaction[col] = lbl.transform(test_transaction[col].astype(str))

In [30]:
from sklearn import preprocessing
for col in train_transaction.columns:
    if  train_transaction[col].dtypes == 'object' :
        lbl = preprocessing.LabelEncoder()
        lbl.fit(train_transaction[col].astype(str))
        train_transaction[col] = lbl.transform(train_transaction[col].astype(str))

In [31]:
train_transaction = reduce_mem_usage(train_transaction)
test_transaction = reduce_mem_usage(test_transaction)

Memory usage of dataframe is 1234.45 MB
Memory usage after optimization is: 328.69 MB
Decreased by 73.4%
Memory usage of dataframe is 1124.93 MB
Memory usage after optimization is: 308.29 MB
Decreased by 72.6%


In [32]:
label = train_transaction['isFraud']
idx = test_transaction['TransactionID']
train = train_transaction.drop(['TransactionID','isFraud'],axis=1)
test  = test_transaction.drop(['TransactionID'],axis=1)

In [33]:
ind_categorical.remove('id_18')

In [34]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# Some useful parameters which will come in handy later on
SEED = 4 # for reproducibility
NFOLDS = 5# set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS, shuffle=False)
i=0
for train_index, test_index in kf.split(train):
    print("TRAIN:", train_index, "TEST:", test_index)


TRAIN: [110824 110825 110826 ... 554115 554116 554117] TEST: [     0      1      2 ... 110821 110822 110823]
TRAIN: [     0      1      2 ... 554115 554116 554117] TEST: [110824 110825 110826 ... 221645 221646 221647]
TRAIN: [     0      1      2 ... 554115 554116 554117] TEST: [221648 221649 221650 ... 332469 332470 332471]
TRAIN: [     0      1      2 ... 554115 554116 554117] TEST: [332472 332473 332474 ... 443292 443293 443294]
TRAIN: [     0      1      2 ... 443292 443293 443294] TEST: [443295 443296 443297 ... 554115 554116 554117]


In [35]:
ntrain=train.shape[0]
ntest=test.shape[0]
print(ntrain, ntest)

554118 506691


In [36]:
del train_transaction, test_transaction

In [37]:
pp = {'verbosity': 1,
 'reg_lambda': 0.5780000000000001,
 'reg_alpha': 1.09,
 'objective': 'binary',
 'num_leaves': 491,
 'n_estimators': 4000,
 'min_data_in_leaf': 138,
 'min_child_weight': 0.019869999999999964,
 'metric': 'auc',
 'max_depth': 15,
 'learning_rate': 0.006,
 'feature_fraction': 0.17100000000000007,
 'boosting_type': 'gbdt',
 'bagging_seed': 1,
 'bagging_fraction': 0.5350000000000001
}

In [38]:
pred_train = np.zeros((ntrain,) )
score = 0
from sklearn import metrics
import lightgbm
i = 1
for train_index, test_index in  kf.split(train):
    
    train_data = lightgbm.Dataset(train.iloc[train_index],label.iloc[train_index], categorical_feature=ind_categorical)
    val_train_data = lightgbm.Dataset(train.iloc[test_index],label.iloc[test_index], categorical_feature=ind_categorical)
    
    model = lightgbm.train(pp,
                               train_data,
                               valid_sets = [val_train_data],
                               num_boost_round=4000,
                               early_stopping_rounds=100,verbose_eval=1)
    
    model.save_model('mode'+str(i)+'.txt')
    pr=model.predict(train.iloc[test_index])
    pred_train[test_index]=pr
    err=metrics.roc_auc_score(label.iloc[test_index], pr)
    score += err*(1/NFOLDS) 
    print("Fold", i, "Точность работы на тестовых данных:",  err)
    i+=1

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's auc: 0.795738
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.839787
[3]	valid_0's auc: 0.84221
[4]	valid_0's auc: 0.855817
[5]	valid_0's auc: 0.860094
[6]	valid_0's auc: 0.862731
[7]	valid_0's auc: 0.868098
[8]	valid_0's auc: 0.869498
[9]	valid_0's auc: 0.870327
[10]	valid_0's auc: 0.875652
[11]	valid_0's auc: 0.876527
[12]	valid_0's auc: 0.878406
[13]	valid_0's auc: 0.878242
[14]	valid_0's auc: 0.878949
[15]	valid_0's auc: 0.879206
[16]	valid_0's auc: 0.878838
[17]	valid_0's auc: 0.879022
[18]	valid_0's auc: 0.880627
[19]	valid_0's auc: 0.880159
[20]	valid_0's auc: 0.880475
[21]	valid_0's auc: 0.880572
[22]	valid_0's auc: 0.880178
[23]	valid_0's auc: 0.880209
[24]	valid_0's auc: 0.880501
[25]	valid_0's auc: 0.8814
[26]	valid_0's auc: 0.881435
[27]	valid_0's auc: 0.881926
[28]	valid_0's auc: 0.881808
[29]	valid_0's auc: 0.881906
[30]	valid_0's auc: 0.882394
[31]	valid_0's auc: 0.882633
[32]	valid_0's auc: 0.882275
[33]	valid_0's auc: 0

[277]	valid_0's auc: 0.902026
[278]	valid_0's auc: 0.902143
[279]	valid_0's auc: 0.902246
[280]	valid_0's auc: 0.902304
[281]	valid_0's auc: 0.902253
[282]	valid_0's auc: 0.902268
[283]	valid_0's auc: 0.902314
[284]	valid_0's auc: 0.902319
[285]	valid_0's auc: 0.90234
[286]	valid_0's auc: 0.902398
[287]	valid_0's auc: 0.902411
[288]	valid_0's auc: 0.902432
[289]	valid_0's auc: 0.902459
[290]	valid_0's auc: 0.902477
[291]	valid_0's auc: 0.902535
[292]	valid_0's auc: 0.902605
[293]	valid_0's auc: 0.90257
[294]	valid_0's auc: 0.902619
[295]	valid_0's auc: 0.902673
[296]	valid_0's auc: 0.902719
[297]	valid_0's auc: 0.902786
[298]	valid_0's auc: 0.902873
[299]	valid_0's auc: 0.902904
[300]	valid_0's auc: 0.902939
[301]	valid_0's auc: 0.902992
[302]	valid_0's auc: 0.903086
[303]	valid_0's auc: 0.903095
[304]	valid_0's auc: 0.9031
[305]	valid_0's auc: 0.903162
[306]	valid_0's auc: 0.90317
[307]	valid_0's auc: 0.903213
[308]	valid_0's auc: 0.903248
[309]	valid_0's auc: 0.903286
[310]	valid_0's

[553]	valid_0's auc: 0.912418
[554]	valid_0's auc: 0.912502
[555]	valid_0's auc: 0.912486
[556]	valid_0's auc: 0.912518
[557]	valid_0's auc: 0.912539
[558]	valid_0's auc: 0.912627
[559]	valid_0's auc: 0.912644
[560]	valid_0's auc: 0.912664
[561]	valid_0's auc: 0.912683
[562]	valid_0's auc: 0.912713
[563]	valid_0's auc: 0.912753
[564]	valid_0's auc: 0.912795
[565]	valid_0's auc: 0.912851
[566]	valid_0's auc: 0.912907
[567]	valid_0's auc: 0.912907
[568]	valid_0's auc: 0.913001
[569]	valid_0's auc: 0.913045
[570]	valid_0's auc: 0.913065
[571]	valid_0's auc: 0.913101
[572]	valid_0's auc: 0.913121
[573]	valid_0's auc: 0.913153
[574]	valid_0's auc: 0.913193
[575]	valid_0's auc: 0.913253
[576]	valid_0's auc: 0.913349
[577]	valid_0's auc: 0.913355
[578]	valid_0's auc: 0.913389
[579]	valid_0's auc: 0.91342
[580]	valid_0's auc: 0.913456
[581]	valid_0's auc: 0.9135
[582]	valid_0's auc: 0.913513
[583]	valid_0's auc: 0.913514
[584]	valid_0's auc: 0.913562
[585]	valid_0's auc: 0.913574
[586]	valid_0

[828]	valid_0's auc: 0.919333
[829]	valid_0's auc: 0.91936
[830]	valid_0's auc: 0.919414
[831]	valid_0's auc: 0.919407
[832]	valid_0's auc: 0.919432
[833]	valid_0's auc: 0.91945
[834]	valid_0's auc: 0.919486
[835]	valid_0's auc: 0.919512
[836]	valid_0's auc: 0.919515
[837]	valid_0's auc: 0.919535
[838]	valid_0's auc: 0.919567
[839]	valid_0's auc: 0.919562
[840]	valid_0's auc: 0.919578
[841]	valid_0's auc: 0.919604
[842]	valid_0's auc: 0.919629
[843]	valid_0's auc: 0.9196
[844]	valid_0's auc: 0.919603
[845]	valid_0's auc: 0.919622
[846]	valid_0's auc: 0.919663
[847]	valid_0's auc: 0.919672
[848]	valid_0's auc: 0.919698
[849]	valid_0's auc: 0.919704
[850]	valid_0's auc: 0.9197
[851]	valid_0's auc: 0.91973
[852]	valid_0's auc: 0.919755
[853]	valid_0's auc: 0.919752
[854]	valid_0's auc: 0.919771
[855]	valid_0's auc: 0.919771
[856]	valid_0's auc: 0.919784
[857]	valid_0's auc: 0.919795
[858]	valid_0's auc: 0.919786
[859]	valid_0's auc: 0.919832
[860]	valid_0's auc: 0.919863
[861]	valid_0's a

...

[551]	valid_0's auc: 0.923382
[552]	valid_0's auc: 0.923413
[553]	valid_0's auc: 0.923404
[554]	valid_0's auc: 0.923456
[555]	valid_0's auc: 0.923488
[556]	valid_0's auc: 0.92354
[557]	valid_0's auc: 0.923547
[558]	valid_0's auc: 0.923547
[559]	valid_0's auc: 0.923568
[560]	valid_0's auc: 0.923613
[561]	valid_0's auc: 0.923634
[562]	valid_0's auc: 0.923669
[563]	valid_0's auc: 0.923683
[564]	valid_0's auc: 0.923706
[565]	valid_0's auc: 0.92371
[566]	valid_0's auc: 0.923731
[567]	valid_0's auc: 0.923745
[568]	valid_0's auc: 0.923757
[569]	valid_0's auc: 0.923804
[570]	valid_0's auc: 0.923825
[571]	valid_0's auc: 0.923806
[572]	valid_0's auc: 0.923868
[573]	valid_0's auc: 0.923898
[574]	valid_0's auc: 0.923909
[575]	valid_0's auc: 0.923948
[576]	valid_0's auc: 0.923997
[577]	valid_0's auc: 0.924022
[578]	valid_0's auc: 0.924018
[579]	valid_0's auc: 0.924025
[580]	valid_0's auc: 0.924023
[581]	valid_0's auc: 0.924036
[582]	valid_0's auc: 0.924094
[583]	valid_0's auc: 0.92412
[584]	valid_0

[826]	valid_0's auc: 0.927939
[827]	valid_0's auc: 0.927934
[828]	valid_0's auc: 0.927952
[829]	valid_0's auc: 0.927989
[830]	valid_0's auc: 0.928008
[831]	valid_0's auc: 0.928012
[832]	valid_0's auc: 0.928033
[833]	valid_0's auc: 0.928033
[834]	valid_0's auc: 0.928036
[835]	valid_0's auc: 0.928065
[836]	valid_0's auc: 0.928065
[837]	valid_0's auc: 0.928081
[838]	valid_0's auc: 0.928094
[839]	valid_0's auc: 0.928092
[840]	valid_0's auc: 0.928097
[841]	valid_0's auc: 0.928092
[842]	valid_0's auc: 0.928096
[843]	valid_0's auc: 0.928134
[844]	valid_0's auc: 0.928135
[845]	valid_0's auc: 0.928134
[846]	valid_0's auc: 0.928134
[847]	valid_0's auc: 0.928145
[848]	valid_0's auc: 0.928167
[849]	valid_0's auc: 0.928182
[850]	valid_0's auc: 0.928196
[851]	valid_0's auc: 0.928218
[852]	valid_0's auc: 0.928246
[853]	valid_0's auc: 0.928245
[854]	valid_0's auc: 0.928262
[855]	valid_0's auc: 0.928292
[856]	valid_0's auc: 0.928304
[857]	valid_0's auc: 0.92832
[858]	valid_0's auc: 0.928318
[859]	valid

[1097]	valid_0's auc: 0.930042
[1098]	valid_0's auc: 0.930071
[1099]	valid_0's auc: 0.930081
[1100]	valid_0's auc: 0.930094
[1101]	valid_0's auc: 0.930102
[1102]	valid_0's auc: 0.930116
[1103]	valid_0's auc: 0.930127
[1104]	valid_0's auc: 0.930139
[1105]	valid_0's auc: 0.930138
[1106]	valid_0's auc: 0.93015
[1107]	valid_0's auc: 0.930137
[1108]	valid_0's auc: 0.930146
[1109]	valid_0's auc: 0.930154
[1110]	valid_0's auc: 0.930152
[1111]	valid_0's auc: 0.930179
[1112]	valid_0's auc: 0.930185
[1113]	valid_0's auc: 0.9302
[1114]	valid_0's auc: 0.930214
[1115]	valid_0's auc: 0.930217
[1116]	valid_0's auc: 0.930216
[1117]	valid_0's auc: 0.930231
[1118]	valid_0's auc: 0.930232
[1119]	valid_0's auc: 0.930223
[1120]	valid_0's auc: 0.93025
[1121]	valid_0's auc: 0.930238
[1122]	valid_0's auc: 0.930255
[1123]	valid_0's auc: 0.930271
[1124]	valid_0's auc: 0.930285
[1125]	valid_0's auc: 0.93028
[1126]	valid_0's auc: 0.930289
[1127]	valid_0's auc: 0.930285
[1128]	valid_0's auc: 0.930284
[1129]	valid_

[1363]	valid_0's auc: 0.931144
[1364]	valid_0's auc: 0.931145
[1365]	valid_0's auc: 0.931151
[1366]	valid_0's auc: 0.931151
[1367]	valid_0's auc: 0.931149
[1368]	valid_0's auc: 0.931156
[1369]	valid_0's auc: 0.931149
[1370]	valid_0's auc: 0.931149
[1371]	valid_0's auc: 0.931152
[1372]	valid_0's auc: 0.931152
[1373]	valid_0's auc: 0.931168
[1374]	valid_0's auc: 0.931175
[1375]	valid_0's auc: 0.931175
[1376]	valid_0's auc: 0.931175
[1377]	valid_0's auc: 0.93118
[1378]	valid_0's auc: 0.931175
[1379]	valid_0's auc: 0.931179
[1380]	valid_0's auc: 0.931183
[1381]	valid_0's auc: 0.931187
[1382]	valid_0's auc: 0.931198
[1383]	valid_0's auc: 0.931183
[1384]	valid_0's auc: 0.93119
[1385]	valid_0's auc: 0.931185
[1386]	valid_0's auc: 0.931174
[1387]	valid_0's auc: 0.931176
[1388]	valid_0's auc: 0.931176
[1389]	valid_0's auc: 0.931173
[1390]	valid_0's auc: 0.931184
[1391]	valid_0's auc: 0.931193
[1392]	valid_0's auc: 0.93119
[1393]	valid_0's auc: 0.931192
[1394]	valid_0's auc: 0.931198
[1395]	vali

[1629]	valid_0's auc: 0.931341
[1630]	valid_0's auc: 0.931351
[1631]	valid_0's auc: 0.931348
[1632]	valid_0's auc: 0.931348
[1633]	valid_0's auc: 0.931354
[1634]	valid_0's auc: 0.931349
[1635]	valid_0's auc: 0.931341
[1636]	valid_0's auc: 0.93135
[1637]	valid_0's auc: 0.931365
[1638]	valid_0's auc: 0.931359
[1639]	valid_0's auc: 0.931364
[1640]	valid_0's auc: 0.931369
[1641]	valid_0's auc: 0.931387
[1642]	valid_0's auc: 0.931393
[1643]	valid_0's auc: 0.931378
[1644]	valid_0's auc: 0.931383
[1645]	valid_0's auc: 0.931376
[1646]	valid_0's auc: 0.931363
[1647]	valid_0's auc: 0.931368
[1648]	valid_0's auc: 0.931363
[1649]	valid_0's auc: 0.931375
[1650]	valid_0's auc: 0.931389
[1651]	valid_0's auc: 0.931385
[1652]	valid_0's auc: 0.93139
[1653]	valid_0's auc: 0.931395
[1654]	valid_0's auc: 0.931394
[1655]	valid_0's auc: 0.93139
[1656]	valid_0's auc: 0.931384
[1657]	valid_0's auc: 0.931383
[1658]	valid_0's auc: 0.931386
[1659]	valid_0's auc: 0.93139
[1660]	valid_0's auc: 0.931403
[1661]	valid

In [39]:
score

0.9359011086653025

In [40]:
wr=1./NFOLDS
predTest = np.zeros((ntest,))
for i in np.arange(NFOLDS):    
    i+=1
    print('loading model'+str(i)+'.txt...')
    bst = lightgbm.Booster(model_file='mode'+str(i)+'.txt')
   
    predTr=bst.predict(test)
    predTest=predTest+predTr*wr

loading model1.txt...
loading model2.txt...
loading model3.txt...
loading model4.txt...
loading model5.txt...


In [41]:
preds = pd.DataFrame({"TransactionID": idx, "isFraud":predTest})
preds.to_csv('submission.csv', index=False)